In [1]:
import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [2]:
# Open the PDF file
pdf_file = open('hh.pdf', 'rb')

In [3]:
import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the question-answering model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Open the PDF file and extract the text
with open('hh.pdf', 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    text = ''
    for page in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page].extract_text()

# Define a function to answer questions
def answer_question(question):
    # Tokenize the input text
    inputs = tokenizer(question, text, return_tensors='pt', max_length=512, truncation=True)
    
    # Get the start and end logits for the answer
    with torch.no_grad():
        start_logits, end_logits = model(**inputs).values()

    # Find the start and end positions with the highest logits
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()
    
    # Extract the answer from the input text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index+1]))
    
    return answer

# Define a function to run the chatbot
def run_chatbot():
    print('Chatbot: Hi, I can answer questions about this PDF file. What would you like to know?')
    while True:
        user_input = input('User: ')
        if user_input.lower() == 'quit':
            print('Chatbot: Goodbye!')
            break
        answer = answer_question(user_input)
        print(f'Chatbot: {answer}')

# Run the chatbot
run_chatbot()


Chatbot: Hi, I can answer questions about this PDF file. What would you like to know?
User: what is applicant age
Chatbot: [SEP]
User: quit
Chatbot: Goodbye!
